In [ ]:
import os
import shutil

from config import SETTINGS

import seabeepy as sb

# Flatten images folders

Some of the `images` folders on `niva-tidy` contain subfolders, which causes problems with the automated workflow. Medyan has said it should be OK to "flatten" everything into a single folder - see e-mail received 11.04.2024 at 12.19.

In [ ]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

In [ ]:
def find_files(base_dir):
    for dirpath, dirnames, filenames in os.walk(base_dir):
        if "config.seabee.yaml" in filenames:
            yield dirpath


def resolve_conflicts_and_move(src, dst, dry_run=True):
    if os.path.isfile(dst):
        base, extension = os.path.splitext(dst)
        i = 1
        while os.path.isfile(dst):
            dst = f"{base}_{i}{extension}"
            i += 1
    if dry_run:
        print(f"Would move {src} to {dst}")
    else:
        # shutil.move(src, dst)
        sb.storage.copy_file(src, dst, minio_client, overwrite=False)
        sb.storage.delete_file(src, minio_client)


def flatten_images_dir(dirpath, dry_run=True):
    images_dir = os.path.join(dirpath, "images")
    if os.path.isdir(images_dir):
        # First, move all files.
        for root, dirs, files in os.walk(images_dir):
            for file in files:
                src = os.path.join(root, file)
                dst = os.path.join(images_dir, file)
                if src != dst:
                    resolve_conflicts_and_move(src, dst, dry_run=dry_run)

        # Then, delete empty directories.
        for root, dirs, files in os.walk(images_dir, topdown=False):
            for dir in dirs:
                dir_path = os.path.join(root, dir)
                if not os.listdir(dir_path):  # Check if directory is empty
                    if dry_run:
                        print(f"Would delete directory {os.path.join(root, dir)}")
                    else:
                        # shutil.rmtree(dir_path)
                        sb.storage.delete_folder(dir_path, minio_client)

In [ ]:
base_dir = r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022"
dry_run = True

# Restructure files
for dirpath in find_files(base_dir):
    # print(dirpath)
    flatten_images_dir(dirpath, dry_run=dry_run)